In [ ]:
import math
import numpy as np
from scipy.io import loadmat
import csv
import random
import pandas as pd
from scipy.ndimage.interpolation import rotate
from sklearn.preprocessing import normalize, scale
import matplotlib
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils import shuffle
from joblib import Parallel, delayed
import multiprocessing
from scipy.misc import imsave
from skimage.io import imread
import pickle

In [2]:
def write_CSV(prediction, fileName):
    header = "Id,Category\n"
    output = ""
    for i in range(len(prediction)):
#         output += str(i) + "," + str(prediction[i]) + '\n'
        output += str(i+1) + "," + str(prediction[i]) + '\n'
    answers = header + output
    with open(fileName, "w") as f:
        f.write(answers)
        
def save_obj(obj, name):
    f = open(name, 'wb')
    pickle.dump(obj, f)
    f.close()

def load_obj(name):
    f = open(name, 'rb')
    obj = pickle.load(f)
    f.close()
    return obj

In [ ]:
## Q1 Data
mnist = loadmat("/Users/clark/Desktop/189Data/hw7_data/mnist_data/images.mat")
# print(mnist.keys())
mnist_data = mnist["images"]
# print(mnist_data.shape)
mnist_data = mnist_data.reshape(28*28, 60000).T
# print(mnist_data.shape)
np.random.shuffle(mnist_data)

## Code for reference

In [ ]:
def k_means(data, k = 10, max_iter = 500):
    length = len(data)
    partition = []
    part_length = []
    for i in range(k):
        partition.append([])
        part_length.append(0)
    #Forgy method: (step 2)
    indices = np.random.randint(low = 0, high=length, size=k)
    mu = []
    count = 0
    for i in indices:
        mu.append(data[i])
    while (True):
        print(count)
        count += 1
        if (count >= max_iter):
            print("Reach max iterations")
            return mu
        
        temp_part = []
        #step 2:
        for i in range(length):
            point = data[i]
            centroid_index = np.argmin(np.linalg.norm(point - mu, axis=1))
            partition[centroid_index].append(point)
        #step 1:
        for i in range(k):
            mu[i] = np.mean(partition[i], axis=0)
            temp_part.append(len(partition[i]))
        
        save_obj(mu, "mu" + str(k)+".pckl")


        if (sum(abs(np.array(temp_part) - np.array(part_length))) >= k):
            part_length = temp_part
        else:
            print("done")
            return mu

In [ ]:
def draw_centroid(centroids):
    length = len(centroids)
    for i in range(length):
        temp = np.array(centroids[i]).reshape(28, 28)
        imsave(str(i) + '.png', temp)
        
def visualization_kmeans(centroids):
    length = len(centroids)
    for i in range(length):
        temp = np.array(centroids[i]).reshape(28, 28)
        plt.imshow(temp)
        plt.show()

In [ ]:
centroids_k5 = k_means(mnist_data, k = 5, max_iter = 500)
centroids_k10 = k_means(mnist_data, k = 10, max_iter = 500)
centroids_k20 = k_means(mnist_data, k = 20, max_iter = 500)

# mu = load_obj("mu" + str(10)+".pckl")

In [ ]:
visualization_kmeans(centroids_k5)
draw_centroid(centroids_k5)

## Low-Rank Approximation

In [ ]:
face = imread("/Users/clark/Desktop/189Data/hw7_data/low-rank_data/face.jpg")
sky = imread("/Users/clark/Desktop/189Data/hw7_data/low-rank_data/sky.jpg")

In [ ]:
def rank_approx(image, rank):
    u,s,v = np.linalg.svd(image)
    #Note: Here, it factors the matrix as u * np.diag(s) * v, so we do not use v.T
    S = np.zeros((u.shape[1], v.shape[0]))
    for i in range(rank):
        S[i][i] = s[i]
    return np.dot(np.dot(u, S), v)
def visualization_rank(image):
    image = image.reshape(face.shape)
    plt.imshow(image, cmap='gray')
    plt.show()

In [ ]:
def mse(orig, low_rank):
    result = np.mean((orig-low_rank) ** 2)
    return result
def plot_mse(image):
    errors = []
    for i in range(1, 101):
        low_rank = rank_approx(image, i)
        error = mse(image, low_rank)
        errors.append(error)
    plt.plot(range(1, 101), errors)
    plt.xlabel("Rank")
    plt.ylabel("Mean Squared Error")
    plt.title("Mean Squared Error vs Rank Graph")
    plt.show()

In [ ]:
rank5 = rank_approx(face, 5)
imsave("face_rank5" + '.png', rank5)
rank20 = rank_approx(face, 20)
imsave("face_rank20" + '.png', rank20)
rank100 = rank_approx(face, 100)
imsave("face_rank100" + '.png', rank100)

In [ ]:
visualization_rank(rank5)
visualization_rank(rank20)
visualization_rank(rank100)

In [ ]:
plot_mse(face)

In [ ]:
rank5 = rank_approx(sky, 5)
imsave("sky_rank5" + '.png', rank5)
rank20 = rank_approx(sky, 20)
imsave("sky_rank20" + '.png', rank20)
rank100 = rank_approx(sky, 100)
imsave("sky_rank100" + '.png', rank100)

In [ ]:
visualization_rank(rank5)
visualization_rank(rank20)
visualization_rank(rank100)

## Joke Recommender Syster

In [3]:
# m: jokes
num_jokes = 100
# n: users
num_users = 24983

### Data Preprocessing

In [4]:
def read_txt(file):
    lst = []
    f = open(file, "r")
    lines = f.readlines()
    for line in lines:
        line = line.split(",")
        lst.append([int(line[0]), int(line[1]), int(line[2])])
    return np.array(lst)

In [5]:
train = loadmat("/Users/clark/Desktop/189Data/hw7_data/joke_data/joke_train.mat")["train"]
#Replace all missing values by zero
train_filled_with_zero = np.nan_to_num(train)
u,s,v = np.linalg.svd(train_filled_with_zero)
validation = read_txt("/Users/clark/Desktop/189Data/hw7_data/joke_data/validation.txt")
test = read_txt("/Users/clark/Desktop/189Data/hw7_data/joke_data/query.txt")

In [91]:
def low_dim_approx(u,s,v, rank):
    #Computation of U,S,V is too slow. Thus, we will only compute it once. 
    #Note: Here, it factors the matrix as u * np.diag(s) * v, so we do not use v.T
    S = np.zeros((u.shape[1], v.shape[0]))
    for i in range(rank):
        S[i][i] = s[i]
    return np.dot(np.dot(u, S), v)

def usv_process(u,s,v,d):
    new_u = np.array([row[:d] for row in u])
    new_v = np.array([row[:d] for row in v.T])
    new_s = np.identity(d)
    for i in range(d):
        new_s[i][i] = s[i]
    return new_u, new_s, new_v

def mse(orig, pred):
    uv = []
    r = []
    for i in range(orig.shape[0]):
        for j in range(orig.shape[1]):
            if not np.isnan(orig[i][j]):
                r.append(orig[i][j])
                uv.append(pred[i][j])
    uv = np.array(uv)
    r = np.array(r)
    #Don't know why, but the equation on pdf seems like it doesn't take the mean.
    #Instead, it takes the sum. I will use the mean here since by definition MSE should be the mean
    result = np.mean((uv-r) ** 2)
    return result

#Recall that <u_i, v_j> approximates R_{ij}
#s = 1 iff rating > 0
def validate(validation, prediction):
    correct = 0.0
    #User index and joke index start at 1
    user_index = [val[0] for val in validation]
    joke_index = [val[1] for val in validation]
    user_like = [val[2] for val in validation]
    length = len(validation)
    for index in range(length):
        i = user_index[index] - 1
        j = joke_index[index] - 1
        s = user_like[index]
        #User like joke:
        if (s == 1 and prediction[i][j] > 0):
            correct += 1
        elif (s == 0 and prediction[i][j] <= 0):
            correct += 1
    return correct / length

In [ ]:
for d in [2, 5, 10, 20]:
    pred = low_dim_approx(u,s,v, d)
    mean_square_error = mse(train, pred)
    accuaracy = validate(validation, pred)
    print("When d is " + str(d) + ", MSE is " + str(mean_square_error) + ", accuracy is " + str(accuaracy) + ".")

In [92]:
def train_and_pred(u, v, data, d = 20, max_iter=1, lamb=350):
    u_shape = u.shape
    v_shape = v.shape
    new_u = u
    new_v = v
    for count in range(max_iter):
        for i in range(u_shape[0]):
            left = lamb * np.identity(d)
            right = np.zeros((d, 1))
            for j in range(v_shape[0]):
                if not np.isnan(data[i][j]):
                    vj = new_v[j].reshape(d, 1)
                    left += np.dot(vj, vj.T)
                    right += data[i][j] * vj
            #Ax = b -> x = A^{-1} b -> numpy.linalg.solve
#             print(np.linalg.inv(left).shape)
#             print(right.shape)
#             print(np.dot(np.linalg.inv(left), right).shape)
#             print(new_u[i].shape)
            new_u[i] = np.dot(np.linalg.inv(left), right).reshape(d,)
        for j in range(v_shape[0]):
            left = lamb * np.identity(d)
            right = np.zeros((d, 1))
            for i in range(u_shape[0]):
                if not np.isnan(data[i][j]):
                    ui = new_u[i].reshape(d, 1)
                    left += np.dot(ui, ui.T)
                    right += data[i][j] * ui
            new_v[j] = np.dot(np.linalg.inv(left), right).reshape(d,)
        print(count)
#         if count % 10 == 0:
#             save_obj(new_u, str(d) + "_u_iter_" + str(count) + ".p")
#             save_obj(new_v, str(d) + "_v_iter_" + str(count) + ".p")
    return np.dot(new_u, new_v.T)

In [ ]:
# def train_with_saved(u_name, v_name, data, d = 20, max_iter=500, lamb=1.1, counts = 0):
#     new_u = np.matrix(load_obj(u_name))
#     new_v = np.matrix(load_obj(v_name))
#     u_shape = new_u.shape
#     v_shape = new_v.shape
#     count = counts
#     while count <= max_iter:
#         for i in range(u_shape[0]):
#             left = np.zeros(v_shape)
#             right = np.zeros((v_shape[0], 1))
#             for temp in range(d):
#                 left[temp][temp] = lamb * 1
#             for j in range(v_shape[0]):
#                 if not np.isnan(data[i][j]):
#                     vj = new_v[j].reshape(v_shape[0], 1)
#                     left += np.dot(vj, vj.T)
#                     right += data[i][j] * vj
#             #Ax = b -> x = A^{-1} b -> numpy.linalg.solve
#             left += 1e-8 * np.identity(v_shape[0])
#             new_u[i] = np.linalg.solve(left, right).T
#         for j in range(v_shape[0]):
#             left = np.zeros((v_shape))
#             right = np.zeros((v_shape[0], 1))
#             for temp in range(d):
#                 left[temp][temp] = lamb * 1
#             for i in range(u_shape[0]):
#                 if not np.isnan(data[i][j]):
#                     ui = new_u[i].reshape(v_shape[0], 1)

#                     left += np.dot(ui, ui.T)
#                     right += data[i][j] * ui
                    
#             left += 1e-8 * np.identity(v_shape[0])
#             new_v[j] = np.linalg.solve(left, right).T
#         if count % 10 == 0:
#             save_obj(new_u, str(d) + "_u_iter_" + str(count) + ".p")
#             save_obj(new_v, str(d) + "_v_iter_" + str(count) + ".p")
#         count += 1
#     return np.dot(new_u, new_v.T)

In [8]:
def predict(approx, test):
    #Each line takes the form “id, i, j”. You are asked to predict if user i likes joke j
    user_id = [t[0] for t in test]
    user_index = [t[1] for t in test]
    joke_index = [t[2] for t in test]
    length = len(user_id)
    s = np.zeros(length, dtype=int)
    for index in range(length):
        i = user_index[index] - 1
        j = joke_index[index] - 1
        #User like joke:
        if (approx[i][j] > 0):
            s[index] = 1
        else:
            s[index] = 0
    return s

## Train lambda

In [142]:
for l in np.arange(-3,4):
    lam = 10 ** l
    ddd= 20
    new_u, new_s, new_v = usv_process(u,s,v,ddd)
    estimate2 = train_and_pred(new_u, new_v, train, ddd, max_iter=5, lamb=lam)
    save_obj(estimate2, "lambda_" + str(lam) + "_5times_"+str(ddd)+".p")
    print("At lambda is " + str(lam) + ", our validation accuracy is " + str(validate(validation, np.array(estimate2))))

0
1
2
3
4
At lambda is 0.001, our validation accuracy is 0.6655826558265583
0
1
2
3
4
At lambda is 0.01, our validation accuracy is 0.67289972899729
0
1
2
3
4
At lambda is 0.1, our validation accuracy is 0.6802168021680217
0
1
2
3
4
At lambda is 1, our validation accuracy is 0.6804878048780488
0
1
2
3
4
At lambda is 10, our validation accuracy is 0.6796747967479675
0
1
2
3
4
At lambda is 100, our validation accuracy is 0.7084010840108401
0
1
2
3
4
At lambda is 1000, our validation accuracy is 0.6685636856368564


In [149]:
for iter_time in np.arange(10, 50, 10):
    lam = 350
    ddd= 20
    new_u, new_s, new_v = usv_process(u,s,v,ddd)
    estimate2 = train_and_pred(new_u, new_v, train, ddd, max_iter=iter_time, lamb=lam)
    save_obj(estimate2, "lambda_" + str(350) + "_" + str(iter_time) + "times_"+str(20)+".p")
    print("When lambda is 350, at d = " + str(ddd) + ", our validation accuracy with " + str(iter_time) + " iteration is " + str(validate(validation, np.array(estimate2))))
    
    dddddd= 15
    new_u, new_s, new_v = usv_process(u,s,v,dddddd)
    estimate3 = train_and_pred(new_u, new_v, train, dddddd, max_iter=iter_time, lamb=lam)
    save_obj(estimate3, "lambda_" + str(350) + "_" + str(iter_time) + "times_"+str(15)+".p")
    print("When lambda is 350, at d = " + str(dddddd) + ", our validation accuracy with " + str(iter_time) + " iteration is " +  str(validate(validation, np.array(estimate3))))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
When lambda is 350, at d = 20, our validation accuracy with 20 iteration is 0.7365853658536585
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
When lambda is 350, at d = 20, our validation accuracy with 20 iteration is 0.7344173441734417
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
When lambda is 350, at d = 20, our validation accuracy with 30 iteration is 0.7357723577235772
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
When lambda is 350, at d = 20, our validation accuracy with 30 iteration is 0.7336043360433604
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
When lambda is 350, at d = 20, our validation accuracy with 40 iteration is 0.7357723577235772
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
When lambda is 350, at d = 20, our validation

KeyboardInterrupt: 

## Train Classifier with best lambda so far

In [104]:
d = 20
new_u, new_s, new_v = usv_process(u,s,v,d)
estimate = train_and_pred(new_u, new_v, train, d, max_iter=50, lamb=500)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [103]:
print(validate(validation, np.array(estimate)))

0.7330623306233063


lamb = 1.1 : 0.6745257452574526


In [105]:
save_obj(estimate, "lambda_500_50times.p")
d = 35
new_u, new_s, new_v = usv_process(u,s,v,d)
estimate1 = train_and_pred(new_u, new_v, train, d, max_iter=50, lamb=300)
save_obj(estimate1, "lambda_300_50times_d_35.p")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [107]:
for ddd in [2, 5, 10, 20]:
    new_u, new_s, new_v = usv_process(u,s,v,ddd)
    estimate2 = train_and_pred(new_u, new_v, train, ddd, max_iter=50, lamb=300)
    save_obj(estimate2, "lambda_300_50times_"+str(ddd)+".p")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


## Test

In [150]:
model = np.array(load_obj("./lambda_350_40times_20.p"))
print(validate(validation, np.array(model)))

0.7357723577235772


In [151]:
naive = predict(model, test)

In [152]:
naive

array([1, 1, 0, ..., 1, 1, 1])

In [153]:
write_CSV(naive, "jokes.txt")